<a href="https://colab.research.google.com/github/harsha-bsm/Topic-Modelling-using-LDA-and-NMF-on-SamsungM30s-reviewsflipkart/blob/master/Preprocessing_and_Modelling_LDA_SAMSUNGM30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import numpy as np
import pandas as pd

In [136]:
data=pd.read_csv("samsungdata.csv")
dataoriginal=pd.read_csv("samsungdata.csv")

In [137]:
data.head()

,Unnamed: 0,Model,rating,user,time_of_review,review,title,user_address,user_type
0,0,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Aryan Patil,2months ago,Bokeh mode is very excellent than any other ha...,Wonderful,Latur District,Certified Buyer
1,1,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,Just awsome product the picture quality is goo...,Great product,North Twenty Four Parganas District,Certified Buyer
2,2,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,4,Suresh Kumar,5months ago,Samsung galaxy m30s is most useful.But Samsung...,Nice product,New Delhi,Certified Buyer
3,3,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,1,Rohit Purba,6months ago,Not worth for me. i buy for my samsung tv this...,Utterly Disappointed,Rajpura,Certified Buyer
4,4,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...",Highly recommended,Muzaffarpur,Certified Buyer


In [138]:
pip install emot 

In [139]:
####### Apply emoticons and emojis

from emot.emo_unicode import UNICODE_EMO, EMOTICONS # contains english text of emojis and emoticons in dict format 
def convert_emojis(review):
  for x in review:
    if x in UNICODE_EMO.keys():
      review=review.replace(x,"")
  return review

In [140]:
#converting all basic contracted foms  
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [141]:
#removing special characters and tabs ,spaces,new lines etc..
def removecharacters(review):
  review= re.sub('[^A-Za-z0-9]+',' ',review)  #anything exept numbers and alphabets, replace them with space
  review=re.sub(r"\n"," ",review)  #new lines into space
  review=re.sub(r"\t"," ",review)  #tabs into space
  review=re.sub(r"\v"," ",review)  #vertical tab into space
  review=re.sub(r"\s"," ",review)   #all extra spaces into single space
  return review  


In [142]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step
# ignore words like won't, haven't etc.. We have already taken care of them

stopword= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not","phone","product","mobile","np"])

In [143]:
from nltk.corpus import stopwords
import nltk

In [144]:
def removestopwords(review):
  return " ".join([word for word in review.split(" ") if not word in stopword])


In [145]:
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))

In [146]:
data["review"]

0      bokeh mode excellent handset budget battery ca...
1      awsome picture quality good battery backup aws...
2      samsung galaxy m30s useful samsung improve sel...
3      worth buy samsung tv live cast smartthins app ...
4      bsolutely best go phonelet clear features1 cam...
                             ...                        
293                                                 okay
294                           super style good look nice
295                                                 good
296                                                 good
297                                                 good
Name: review, Length: 298, dtype: object

In [147]:
data["review"].to_csv("samsung_preprocessed.csv")

In [148]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import ldamodel
import gensim.corpora
from sklearn.decomposition import LatentDirichletAllocation

In [149]:
#https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
  topics_cluster=[]
  for topic_idx, topic in enumerate(model.components_):
      message = "Topic #%d: " % topic_idx
      message += ",".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
      topics_cluster.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
  return topics_cluster
  

### BOW and LDA 

In [153]:
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
topics=print_top_words(lda,bow.get_feature_names(),5)

In [154]:
lda.fit_transform(df_bow)

array([[0.00484783, 0.9853934 , 0.00488656, 0.00487221],
       [0.00665282, 0.98002853, 0.00663327, 0.00668537],
       [0.00554561, 0.00562724, 0.00556941, 0.98325775],
       ...,
       [0.12634234, 0.12995352, 0.12660252, 0.61710162],
       [0.12634234, 0.12995352, 0.12660252, 0.61710162],
       [0.12634234, 0.12995352, 0.12660252, 0.61710162]])

In [155]:
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.004848,0.985393,0.004887,0.004872,1,Bokeh mode is very excellent than any other ha...,"[good, battery, nice, camera, super]"
1,0.006653,0.980029,0.006633,0.006685,1,Just awsome product the picture quality is goo...,"[good, battery, nice, camera, super]"
2,0.005546,0.005627,0.005569,0.983258,3,Samsung galaxy m30s is most useful.But Samsung...,"[good, nice, camera, samsung, superb]"
3,0.003935,0.003962,0.003962,0.988141,3,Not worth for me. i buy for my samsung tv this...,"[good, nice, camera, samsung, superb]"
4,0.005984,0.005866,0.005780,0.982370,3,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[good, nice, camera, samsung, superb]"
...,...,...,...,...,...,...,...
293,0.125072,0.125068,0.624801,0.125059,2,okay,"[best, good, camera, range, battery]"
294,0.025247,0.923319,0.025080,0.026353,1,Super style and good look nice product,"[good, battery, nice, camera, super]"
295,0.126342,0.129954,0.126603,0.617102,3,good,"[good, nice, camera, samsung, superb]"
296,0.126342,0.129954,0.126603,0.617102,3,good,"[good, nice, camera, samsung, superb]"


#### LDA on topics obtained

In [20]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['buy', 'online', 'performance', 'buy online'],
 ['good', 'best', 'nice', 'xcellent'],
 ['battery', 'camera', 'samsung', 'wesome'],
 ['thanks', 'price', 'hangs', 'awsome']]

### TFIDF AND LDA

In [21]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,8)

[['money',
  'value money',
  'value',
  'okay',
  'super nice',
  'flipkart',
  'wesome value',
  'things good'],
 ['good',
  'nice',
  'superb',
  'battery',
  'battery backup',
  'camera',
  'backup',
  'backup good'],
 ['xcellent',
  'awsm',
  'waysome',
  'utstanding',
  'nice awsm',
  'easy',
  'smooth easy',
  'battery'],
 ['loved',
  'rattan',
  'worth money',
  'worth',
  'money',
  'camera battery',
  'good one',
  'nice cell'],
 ['love',
  'thanks',
  'nice',
  'display',
  'excellent',
  'super',
  'xcellent',
  'flipkart'],
 ['wesome', 'bad', 'comments', 'camara', 'poor', 'use', 'expected', 'thank'],
 ['super',
  'smartphone',
  'superrrrbbbbb',
  'sm',
  'buy',
  'flipkart',
  'nice products',
  'xcellent smartphone'],
 ['ok', 'price', 'suprrrrrrrrr', 'wsm', 'best', 'set', 'like', 'good price'],
 ['camera',
  'fantastic',
  'goods',
  'fresh',
  'ear',
  'good',
  'wesome',
  'super camera'],
 ['best',
  'awesome',
  'good',
  'samsung',
  'good samsung',
  'great',
  'wo

##### LDA on topics obtained 

In [22]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['nice', 'love', 'buy', 'camera'],
 ['money', 'best', 'value', 'rattan'],
 ['good', 'super', 'samsung', 'nice'],
 ['worth', 'awsm', 'excellent', 'superrrrbbbbb']]

### NMF - TFIDF

In [23]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [24]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)


In [25]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,6)

[['good', 'one', 'normal', 'choic', 'isplay', 'choice'],
 ['nice', 'awsm', 'phones', 'samsang', 'cell', 'osm'],
 ['super', 'quality', 'camera', 'also', 'performance', 'love'],
 ['wesome', 'outstanding', 'perfomance', 'loving', 'slow', 'gaming'],
 ['best', 'samsung', 'price', 'loved', 'range', 'buy'],
 ['battery', 'camera', 'display', 'backup', 'poor', 'back'],
 ['superb', 'expected', 'great', 'got', 'galaxy', 'tq'],
 ['xcellent', 'flipkart', 'smartphone', 'love', 'excellent', 'thanks'],
 ['money', 'awesome', 'value', 'much', 'worth', 'delivery'],
 ['bad', 'ok', 'thank', 'worst', 'working', 'hangs']]

#### NMF on topics obtained

In [26]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]

In [27]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'also', 'wesome', 'one'],
 ['quality', 'poor', 'worst', 'working'],
 ['xcellent', 'smartphone', 'cell', 'choic'],
 ['flipkart', 'awesome', 'backup', 'normal'],
 ['loving', 'money', 'awsm', 'worth']]

### NMF - BOW

In [120]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
bow_words=bow.get_feature_names()
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)

print_top_words(nmf,bow_words,4)

[['phone', 'battery', 'best', 'camera'],
 ['good', 'camera', 'samsung', 'price'],
 ['product', 'samsung', 'wesome', 'buy'],
 ['nice', 'mobile', 'phone', 'worst']]

In [124]:
topics=print_top_words(nmf,bow_words,5)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(df_bow),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.92,0.07,0.00,0.00,0,Bokeh mode is very excellent than any other ha...,"[phone, battery, best, camera, display]"
1,0.28,0.07,0.87,0.00,2,Just awsome product the picture quality is goo...,"[product, samsung, wesome, buy, flipkart]"
2,0.71,0.40,0.00,0.00,0,Samsung galaxy m30s is most useful.But Samsung...,"[phone, battery, best, camera, display]"
3,0.00,0.00,2.05,0.01,2,Not worth for me. i buy for my samsung tv this...,"[product, samsung, wesome, buy, flipkart]"
4,0.88,0.00,0.00,0.00,0,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[phone, battery, best, camera, display]"
...,...,...,...,...,...,...,...
293,0.04,0.00,0.00,0.00,0,okay,"[phone, battery, best, camera, display]"
294,0.04,0.07,0.06,0.26,3,Super style and good look nice product,"[nice, mobile, phone, worst, delivery]"
295,0.00,0.07,0.00,0.00,1,good,"[good, camera, samsung, price, performance]"
296,0.00,0.07,0.00,0.00,1,good,"[good, camera, samsung, price, performance]"


In [78]:
np.round(nmf.transform(bow_review),2).shape

(298, 10)

In [80]:
bow_review.shape

(298, 234)

In [79]:
nmf.components_.shape

(10, 234)

In [85]:
topics=print_top_words(nmf,bow_words,)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(df_bow),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]

#### NMF on topics obtained ------------------------------------------

In [119]:
### extrating topics
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=3, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['battery', 'flipkart', 'also', 'power'],
 ['camera', '12', '2020', 'quality'],
 ['buy', '3rd', 'better', 'competition']]

In [95]:
topics=print_top_words(nmf,bow_words,4)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(df_bow),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic#[["review","dominant_topic_keywords","dominant_topic"]]

,topic_0,topic_1,topic_2,dominant_topic,review,dominant_topic_keywords
0,0.00,0.28,0.00,1,Bokeh mode is very excellent than any other ha...,"[xcellent, value, quality, performance]"
1,0.00,0.00,0.25,2,Just awsome product the picture quality is goo...,"[experience, backup, use, budget]"
2,0.00,0.00,0.25,2,Samsung galaxy m30s is most useful.But Samsung...,"[experience, backup, use, budget]"
3,0.28,0.00,0.00,0,Not worth for me. i buy for my samsung tv this...,"[love, hand, battery, superb]"
4,0.28,0.00,0.00,0,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[love, hand, battery, superb]"
5,0.00,0.00,0.25,2,I have realized that this phone is full packag...,"[experience, backup, use, budget]"
6,0.28,0.00,0.00,0,"one of the good phone in this price range , th...","[love, hand, battery, superb]"
7,0.00,0.00,0.25,2,Nice mobile and it comes good battery back up....,"[experience, backup, use, budget]"
8,0.00,0.00,0.25,2,Been using this phone for more than 4 months n...,"[experience, backup, use, budget]"
9,0.00,0.28,0.00,1,This phone is works excellent so far after (6 ...,"[xcellent, value, quality, performance]"


## POS TAGGING taking only nouns

In [125]:
  import nltk
  nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [126]:
def only_nouns(review):
  lis= " ".join([each.strip(" ") for each,pos in nltk.pos_tag(review.split()) if pos=="NN"])
  return lis
  


In [156]:
data["review"]=data["review"].apply(lambda x:only_nouns(x))

### LDA BOW ---  (topipc extraction/keyword extraction)

In [190]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),20)

[['price',
  'battery',
  'quality',
  'range',
  'bit',
  'buy',
  'mark',
  'camera',
  'performance',
  'power',
  'network',
  'experience',
  'button',
  'voice',
  'finger',
  'print',
  'lagsnetwork',
  'month',
  'amazingperformance',
  'none'],
 ['battery',
  'camera',
  'backup',
  'performance',
  'budget',
  'smartphone',
  'handset',
  'hand',
  'range',
  'experience',
  'gaming',
  'display',
  'samsung',
  'good',
  'bokeh',
  'features1',
  'phonelet',
  'charger3',
  'dispay',
  'phon'],
 ['superb',
  'samsung',
  'flipkart',
  'delivery',
  'm30s',
  'super',
  'galaxy',
  'backup',
  'camara',
  'selfie',
  'bugdet',
  'mar',
  'gui',
  'priceworthy',
  'beatty',
  'oshawa',
  'service',
  'effort',
  'thks',
  'device'],
 ['battery',
  'wesome',
  'display',
  'camera',
  'life',
  'mah',
  'money',
  'isplay',
  'hand',
  'love',
  'glass',
  'use',
  'package',
  'rupee',
  'ram',
  'drain',
  'work',
  'day',
  'management',
  'feel'],
 ['super',
  'use',
  'xce

In [191]:
topics=print_top_words(lda,bow.get_feature_names(),6)
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,dominant_topic,review,dominant_topic_keywords
0,0.006667,0.939996,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006668,0.006668,1,Bokeh mode is very excellent than any other ha...,"[battery, camera, backup, performance, budget,..."
1,0.909989,0.010004,0.010001,0.010001,0.010001,0.010001,0.010000,0.010000,0.010001,0.010002,0,Just awsome product the picture quality is goo...,"[price, battery, quality, range, bit, buy]"
2,0.008334,0.008336,0.924983,0.008335,0.008335,0.008334,0.008333,0.008333,0.008337,0.008339,2,Samsung galaxy m30s is most useful.But Samsung...,"[superb, samsung, flipkart, delivery, m30s, su..."
3,0.007143,0.007143,0.007144,0.007143,0.007143,0.935712,0.007143,0.007143,0.007143,0.007143,5,Not worth for me. i buy for my samsung tv this...,"[online, buy, call, function, time, purchase]"
4,0.008334,0.924997,0.008333,0.008335,0.008333,0.008333,0.008333,0.008333,0.008334,0.008334,1,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[battery, camera, backup, performance, budget,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,0.050000,0.050000,0.050000,4,okay,"[super, use, xcellent, performance, experience..."
294,0.033333,0.033337,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.699996,0.033333,8,Super style and good look nice product,"[display, battery, camera, problem, ok, price]"
295,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0,good,"[price, battery, quality, range, bit, buy]"
296,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0,good,"[price, battery, quality, range, bit, buy]"


In [192]:
modelcomponents=lda.components_/(np.sum(lda.components_,axis=1)).reshape(-1,1)
pd.DataFrame(modelcomponents,index=topic,columns=bow.get_feature_names())

,ability,amazing,amazingperformance,amoled4,android,anything,apple,awesome,awsm,awsome,backup,battery,battry,beast,beatty,bettery,bit,body,bokeh,box,bro,budget,bugdet,butter,button,buy,call,cam,camara,camera,capture,cast,cell,chalta,charge,charger,charger3,choic,choice,coin,...,sm,smart,smartphone,software,sooooooperb,speaker,stabilization,state,style,super,superb,superrrrbbbbb,suprrrrrrrrr,switch,task,team,term,thakarthu,thank,thing,thks,till,time,touch,tq,tunney,tv,use,value,verall,video,voice,waysome,wesome,wid,work,worth,worthless,wsm,xcellent
topic_0,0.013302,0.001209,0.013302,0.001209,0.001209,0.001209,0.001209,0.001209,0.013302,0.013302,0.013302,0.040337,0.001209,0.001209,0.001209,0.001209,0.025395,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.013302,0.025395,0.001209,0.001209,0.001209,0.025393,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,...,0.013302,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.013302,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.013301,0.001209,0.001209,0.001209,0.013302,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209,0.001209
topic_1,0.000853,0.009385,0.000853,0.009385,0.000853,0.009385,0.000853,0.000853,0.000853,0.000853,0.051221,0.133712,0.009385,0.009385,0.000853,0.000853,0.000853,0.000853,0.009385,0.000853,0.000853,0.026450,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.071924,0.000853,0.000853,0.000853,0.009385,0.009385,0.000853,0.009385,0.009385,0.000853,0.000853,...,0.000853,0.000853,0.026450,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.009385,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.009385,0.000853,0.009385,0.000853,0.000853,0.000853,0.009385,0.000853,0.000853,0.000853,0.009385,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853,0.000853
topic_2,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.016506,0.001379,0.001379,0.001379,0.015173,0.001379,0.001379,0.001379,0.001379,0.001379,0.015173,0.001379,0.015173,0.001379,0.001379,0.001379,0.001379,0.001379,0.015173,0.015172,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,...,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001380,0.001379,0.028967,0.125522,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.015173,0.001379,0.001379,0.001379,0.001382,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379,0.001379
topic_3,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.092947,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.035566,0.012247,0.001113,0.012247,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,...,0.001113,0.012247,0.001113,0.012247,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.001113,0.012247,0.001113,0.001113,0.012247,0.001113,0.001113,0.001113,0.001113,0.012247,0.001113,0.001113,0.001113,0.001113,0.001113,0.090184,0.001113,0.012247,0.012245,0.001113,0.001113,0.001113
topic_4,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.015625,0.001420,0.001420,0.001420,0.015625,0.001420,0.001420,0.001420,0.001420,0.015625,0.001420,0.001420,0.001420,0.001420,0.001420,0.015625,0.001420,0.001420,0.001420,0.001420,0.001420,0.015625,0.001420,0.001421,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,...,0.001420,0.001420,0.015625,0.001420,0.015625,0.015625,0.001420,0.001420,0.001420,0.086648,0.001420,0.001420,0.001420,0.001420,0.001420,0.001420,0.

#### (LDA again - on topics obtained) 

In [193]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['battery', 'camera', 'delivery', 'online'],
 ['performance', 'excellent', 'problem', 'osm'],
 ['bit', 'nice', 'thank', 'value'],
 ['quality', 'display', 'box', 'backup']]

In [194]:
topics=print_top_words(lda,bow.get_feature_names(),4)
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.125119,0.125211,0.624471,0.125199,2,Bokeh mode is very excellent than any other ha...,"[bit, nice, thank, value]"
1,0.624862,0.125050,0.125040,0.125047,0,Just awsome product the picture quality is goo...,"[battery, camera, delivery, online]"
2,0.125046,0.125081,0.125065,0.624809,3,Samsung galaxy m30s is most useful.But Samsung...,"[quality, display, box, backup]"
3,0.125102,0.125181,0.125146,0.624572,3,Not worth for me. i buy for my samsung tv this...,"[quality, display, box, backup]"
4,0.125119,0.125211,0.624471,0.125199,2,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[bit, nice, thank, value]"
5,0.624862,0.125050,0.125040,0.125047,0,I have realized that this phone is full packag...,"[battery, camera, delivery, online]"
6,0.624826,0.125063,0.125051,0.125060,0,"one of the good phone in this price range , th...","[battery, camera, delivery, online]"
7,0.125102,0.125181,0.125146,0.624572,3,Nice mobile and it comes good battery back up....,"[quality, display, box, backup]"
8,0.125028,0.624887,0.125039,0.125046,1,Been using this phone for more than 4 months n...,"[performance, excellent, problem, osm]"
9,0.125119,0.125211,0.624471,0.125199,2,This phone is works excellent so far after (6 ...,"[bit, nice, thank, value]"


### LDA TFIDF

In [36]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=8,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,4)

[['awesome', 'excellent', 'isplay', 'sm'],
 ['camera', 'super', 'quality', 'm30s'],
 ['superb', 'samsung', 'delivery', 'love'],
 ['battery', 'price', 'wesome', 'backup'],
 ['ok', 'smartphone', 'use', 'purchase'],
 ['money', 'value', 'call', 'delivery'],
 ['nice', 'experience', 'function', 'thank'],
 ['ear', 'perfomance', 'android', 'butter']]

#### (LDA again - on topics obtained) 

In [37]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['super', 'ear', 'glass', 'butter'],
 ['delivery', 'm30s', 'use', 'rattan'],
 ['camera', 'display', 'isplay', 'wesome'],
 ['experience', 'ok', 'thank', 'mi']]

### NMF BOW (better results)

In [128]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])


In [133]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,5)

[['phone', 'superb', 'use', 'range', 'battery'],
 ['product', 'delivery', 'buy', 'function', 'team'],
 ['battery', 'display', 'camera', 'price', 'performance'],
 ['mobile', 'samsung', 'super', 'performance', 'flipkart'],
 ['camera', 'quality', 'money', 'wesome', 'value']]

In [134]:
topics=print_top_words(nmf,bow_words,5)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(bow_review),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,topic_4,dominant_topic,review,dominant_topic_keywords
0,0.00,0.00,1.14,0.00,0.0,2,Bokeh mode is very excellent than any other ha...,"[battery, display, camera, price, performance]"
1,0.21,0.57,0.10,0.00,0.3,1,Just awsome product the picture quality is goo...,"[product, delivery, buy, function, team]"
2,0.18,0.00,0.17,0.70,0.0,3,Samsung galaxy m30s is most useful.But Samsung...,"[mobile, samsung, super, performance, flipkart]"
3,0.00,1.52,0.00,0.23,0.0,1,Not worth for me. i buy for my samsung tv this...,"[product, delivery, buy, function, team]"
4,0.09,0.00,0.90,0.00,0.0,2,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[battery, display, camera, price, performance]"
...,...,...,...,...,...,...,...,...
293,0.00,0.00,0.00,0.00,0.0,0,okay,"[phone, superb, use, range, battery]"
294,0.00,0.34,0.00,0.00,0.0,1,Super style and good look nice product,"[product, delivery, buy, function, team]"
295,0.00,0.00,0.00,0.00,0.0,0,good,"[phone, superb, use, range, battery]"
296,0.00,0.00,0.00,0.00,0.0,0,good,"[phone, superb, use, range, battery]"


#### (NMF again - on topics obtained) 

In [132]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['camera', 'cost', 'delivery', 'mark'],
 ['life', 'voice', 'wesome', 'backup'],
 ['performance', 'awesome', 'quality', 'ok'],
 ['battery', 'buy', 'flipkart', 'display'],
 ['xcellent', 'price', 'awsm', 'box']]

### NMF TFIDF

In [50]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [51]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,4)

[['battery', 'backup', 'camera', 'performance'],
 ['camera', 'quality', 'experience', 'light'],
 ['wesome', 'life', 'model', 'battery'],
 ['price', 'range', 'segment', 'mid'],
 ['super', 'flipkart', 'purchase', 'use'],
 ['superb', 'delivery', 'condition', 'galaxy'],
 ['money', 'value', 'time', 'delivery'],
 ['samsung', 'love', 'm30s', 'performance'],
 ['display', 'ok', 'xcellent', 'smartphone'],
 ['nice', 'awesome', 'osm', 'mi']]

In [53]:
topics=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(nmf.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,dominant_topic,review,dominant_topic_keywords
0,0.0,0.000000,0.0,0.000000,0.599101,0.0,0.000000,0.0,0.000000,0.0,4,Bokeh mode is very excellent than any other ha...,topic_4
1,0.0,0.761225,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,1,Just awsome product the picture quality is goo...,topic_1
2,0.0,0.000000,0.0,0.763039,0.000000,0.0,0.000000,0.0,0.000000,0.0,3,Samsung galaxy m30s is most useful.But Samsung...,topic_3
3,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.382495,0.0,8,Not worth for me. i buy for my samsung tv this...,topic_8
4,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.329293,0.0,0.000000,0.0,6,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...",topic_6


#### (NMF again - on topics obtained) ---------------------------------------------------

In [43]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'model', 'money', 'smartphone'],
 ['performance', 'm30s', 'time', 'nice'],
 ['delivery', 'keybad', 'galaxy', 'ok'],
 ['battery', 'xcellent', 'quality', 'backup']]

## POS TAG NOUNS and ADJECTIVES

In [82]:
data=pd.read_csv("samsungdata.csv")
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))

In [83]:
def only_nouns_adjectives(review):
  lis= " ".join([each.strip(" ") for each,pos in nltk.pos_tag(review.split()) if (pos=="NN" or pos=="JJ")])
  return lis
data["title"]=data["title"].apply(lambda x:only_nouns_adjectives(x))


### LDA BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)

[['flipkart', 'best', 'value', 'worst'],
 ['super', 'awsome', 'quality', 'fine'],
 ['hours', 'display', 'battery', 'great'],
 ['best', 'camera', 'good', 'bad'],
 ['hangs', 'no', 'thanks', 'flipkart'],
 ['poor', 'excellent', 'good', 'box'],
 ['superb', 'samsung', 'buy', 'smartphone'],
 ['good', 'battery', 'camera', 'performance'],
 ['xcellent', 'awesome', 'money', 'worth'],
 ['nice', 'best', 'wesome', 'samsung']]

#### LDA again

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['fine', 'best', 'superb', 'hangs'],
 ['worth', 'battery', 'price', 'smartphone'],
 ['money', 'camera', 'good', 'poor'],
 ['hours', 'buy', 'xcellent', 'price'],
 ['great', 'box', 'samsung', 'flipkart']]

### LDA TFIDF

In [ ]:

tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,5)

[['display', 'performance', 'best', 'battery', 'nicc'],
 ['super', 'awesome', 'flipkart', 'value', 'money'],
 ['thanks', 'suprrrrrrrrr', 'cell', 'tanks', 'perform'],
 ['bad', 'awsm', 'great', 'loving', 'prublam'],
 ['no', 'hangs', 'smooth', 'frequently', 'lovely'],
 ['exellent', 'sm', 'nice', 'isplay', 'life'],
 ['xcellent', 'superb', 'love', 'smartphone', 'flipkart'],
 ['good', 'camera', 'price', 'quality', 'nice'],
 ['backup', 'battery', 'camara', 'worth', 'loved'],
 ['nice', 'wesome', 'best', 'samsung', 'ok']]

#### LDA again

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['flipkart', 'backup', 'tanks', 'loving'],
 ['suprrrrrrrrr', 'xcellent', 'sm', 'bad'],
 ['battery', 'worth', 'love', 'frequently'],
 ['nice', 'best', 'value', 'money']]

### NMF BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['good', 'battery', 'camera', 'display'],
 ['super', 'superb', 'great', 'display'],
 ['samsung', 'buy', 'm30s', 'loved'],
 ['nice', 'delivery', 'thanks', 'awsm'],
 ['quality', 'camera', 'hangs', 'using'],
 ['best', 'price', 'range', 'camera'],
 ['wesome', 'battery', 'money', 'xcellent'],
 ['performance', 'excellent', 'no', 'display'],
 ['use', 'ok', 'best', 'awsome'],
 ['flipkart', 'thank', 'love', 'worst']]

#### NMF again

In [ ]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['display', 'hours', 'buy', 'worst'],
 ['battery', 'great', 'money', 'good'],
 ['camera', 'xcellent', 'online', 'backup'],
 ['best', 'quality', 'budget', 'hangs'],
 ['performance', 'love', 'samsung', 'better']]

### NMF TFIDF

In [85]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [86]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,6)

[['good', 'one', 'normal', 'choic', 'isplay', 'choice'],
 ['nice', 'awsm', 'phones', 'samsang', 'cell', 'osm'],
 ['super', 'quality', 'camera', 'also', 'performance', 'love'],
 ['wesome', 'outstanding', 'perfomance', 'loving', 'slow', 'gaming'],
 ['best', 'samsung', 'price', 'loved', 'range', 'buy'],
 ['battery', 'camera', 'display', 'backup', 'poor', 'back'],
 ['superb', 'expected', 'great', 'got', 'galaxy', 'tq'],
 ['xcellent', 'flipkart', 'smartphone', 'love', 'excellent', 'thanks'],
 ['money', 'awesome', 'value', 'much', 'worth', 'delivery'],
 ['bad', 'ok', 'thank', 'worst', 'working', 'hangs']]

#### NMF again -------------

In [88]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['xcellent', 'expected', 'loved', 'bad'],
 ['loving', 'great', 'samsung', 'price'],
 ['excellent', 'samsang', 'bad', 'camera'],
 ['awsm', 'superb', 'slow', 'nice']]

In [89]:
topics=print_top_words(nmf,bow_words,4)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(nmf.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.000000,0.453485,0.000000,0.000000,1,Bokeh mode is very excellent than any other ha...,"[delivery, buy, great, excellent]"
1,0.000000,0.453485,0.000000,0.000000,1,Just awsome product the picture quality is goo...,"[delivery, buy, great, excellent]"
2,0.385215,0.000000,0.000000,0.000000,0,Samsung galaxy m30s is most useful.But Samsung...,"[loved, battery, camera, awsome]"
3,0.000000,0.000000,0.357684,0.000000,2,Not worth for me. i buy for my samsung tv this...,"[bad, good, awsome, backup]"
4,0.000000,0.000000,0.000000,0.357258,3,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[awsm, look, hangs, display]"
5,0.000000,0.000000,0.000000,0.357258,3,I have realized that this phone is full packag...,"[awsm, look, hangs, display]"
6,0.000000,0.453485,0.000000,0.000000,1,"one of the good phone in this price range , th...","[delivery, buy, great, excellent]"
7,0.000000,0.000000,0.357684,0.000000,2,Nice mobile and it comes good battery back up....,"[bad, good, awsome, backup]"
8,0.000000,0.000000,0.357684,0.000000,2,Been using this phone for more than 4 months n...,"[bad, good, awsome, backup]"
9,0.000000,0.000000,0.357684,0.000000,2,This phone is works excellent so far after (6 ...,"[bad, good, awsome, backup]"


# Sentence Tokenisation

In [90]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [91]:
review=pd.read_csv("samsungdata.csv")["review"].values

In [92]:
import nltk.data

In [93]:
sent=[]
for i in review:
  sent.extend(nltk.sent_tokenize(i))

In [94]:
len(sent)

348

In [95]:
df=pd.DataFrame(sent,columns=["review"])

In [96]:
df.head()

,review
0,Bokeh mode is very excellent than any other ha...
1,Just awsome product the picture quality is goo...
2,I reccomend you to buy this product
3,Samsung galaxy m30s is most useful.But Samsung...
4,Where is i am disappointed with the selfie but...


In [97]:
df.shape

(348, 1)

In [98]:
df["review"]=df["review"].apply(lambda x:convert_emojis(x))
df["review"]=df["review"].apply(lambda x:x.lower()) 
df["review"]=df["review"].apply(lambda x:decontracted(x))
df["review"]=df["review"].apply(lambda x:removecharacters(x))
df["review"]=pd.DataFrame(df["review"].apply(lambda x:removestopwords(x)))

In [99]:
#data["title"]=data["title"].apply(lambda x:only_nouns_adjectives(x))
#df["review"]=df["review"].apply(lambda x:only_nouns(x))

##sent tokanised 

### BOW LDA

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(df["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),5)

[['wesome', 'flipkart', 'worst', 'ok', 'use'],
 ['super', 'samsung', 'quality', 'bad', 'camera'],
 ['xcellent', 'excellent', 'battery', 'thanks', 'camera'],
 ['good', 'camera', 'battery', 'super', 'one'],
 ['hangs', 'best', 'no', 'frequently', 'smooth'],
 ['good', 'poor', 'buy', 'online', 'though'],
 ['superb', 'good', 'like', 'buy', 'performance'],
 ['good', 'battery', 'camera', 'display', 'money'],
 ['battery', 'worth', 'good', 'backup', 'money'],
 ['nice', 'best', 'awesome', 'battery', 'price']]

#### LDA again

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['money', 'ok', 'performance', 'bad'],
 ['price', 'though', 'worth', 'smooth'],
 ['battery', 'good', 'hangs', 'samsung'],
 ['camera', 'buy', 'nice', 'flipkart'],
 ['best', 'super', 'wesome', 'online']]

### TFIDF LDA

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(df["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,5)

[['wesome', 'ok', 'smart', 'months', 'quality'],
 ['super', 'quality', 'love', 'flipkart', 'samsung'],
 ['excellent', 'great', 'loved', 'thank', 'camera'],
 ['xcellent', 'bad', 'battery', 'camera', 'one'],
 ['samsung', 'hangs', 'no', 'frequently', 'smooth'],
 ['recommend', 'worthless', 'exellent', 'sm', 'online'],
 ['superb', 'smartphone', 'sad', 'buy', 'good'],
 ['good', 'camera', 'battery', 'display', 'also'],
 ['worth', 'normal', 'money', 'wsm', 'camera'],
 ['nice', 'best', 'awesome', 'battery', 'price']]

#### (LDA AGAIN)

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['hangs', 'bad', 'also', 'worth'],
 ['camera', 'battery', 'samsung', 'one'],
 ['quality', 'thank', 'sm', 'smartphone'],
 ['good', 'smooth', 'frequently', 'superb']]

### NMF TFIDF

In [ ]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,5)

[['camera', 'flipkart', 'xcellent', 'months', 'money'],
 ['battery', 'xcellent', 'flipkart', 'months', 'money'],
 ['good', 'xcellent', 'flipkart', 'months', 'money'],
 ['quality', 'bad', 'xcellent', 'flipkart', 'money'],
 ['samsung', 'xcellent', 'flipkart', 'money', 'loved'],
 ['no', 'one', 'awesome', 'loved', 'superb'],
 ['xcellent', 'buy', 'online', 'frequently', 'sad'],
 ['excellent', 'normal', 'best', 'great', 'smooth'],
 ['also', 'smartphone', 'love', 'price', 'recommend'],
 ['worthless', 'wsm', 'ok', 'super', 'hangs']]

### NMF BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['good', 'battery', 'camera', 'display'],
 ['super', 'superb', 'great', 'display'],
 ['samsung', 'buy', 'm30s', 'loved'],
 ['nice', 'delivery', 'thanks', 'awsm'],
 ['quality', 'camera', 'hangs', 'using'],
 ['best', 'price', 'range', 'camera'],
 ['wesome', 'battery', 'money', 'xcellent'],
 ['performance', 'excellent', 'no', 'display'],
 ['use', 'ok', 'best', 'awsome'],
 ['flipkart', 'thank', 'love', 'worst']]

#### nmf again

In [ ]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['display', 'hours', 'buy', 'worst'],
 ['battery', 'great', 'money', 'good'],
 ['camera', 'xcellent', 'online', 'backup'],
 ['best', 'quality', 'budget', 'hangs'],
 ['performance', 'love', 'samsung', 'better']]

## POS TAGGING ONLY NOUNS

In [103]:
df["review"]=df["review"].apply(lambda x:only_nouns(x))

### BOW LDA

In [133]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(df["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)

[['super', 'wesome', 'price', 'camera'],
 ['superb', 'buy', 'awesome', 'use'],
 ['camera', 'display', 'samsung', 'battery'],
 ['battery', 'backup', 'money', 'ok']]

In [134]:
topics=print_top_words(lda,bow.get_feature_names(),4)
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=df["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic[:11]

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.016826,0.016841,0.949477,0.016856,2,bokeh mode handset budget battery camera displ...,"[camera, display, samsung, battery]"
1,0.032914,0.032456,0.891801,0.042829,2,picture quality battery backup awsome pub g ex...,"[camera, display, samsung, battery]"
2,0.083374,0.749892,0.083363,0.083372,1,reccomend buy,"[superb, buy, awesome, use]"
3,0.036633,0.035741,0.891825,0.035801,2,samsung galaxy m30s samsung selfie camera,"[camera, display, samsung, battery]"
4,0.182244,0.035891,0.746052,0.035813,2,selfie bugdet performance display super decent,"[camera, display, samsung, battery]"
5,0.125012,0.624969,0.125009,0.125011,1,superb,"[superb, buy, awesome, use]"
6,0.125119,0.125148,0.624633,0.125100,2,worth,"[camera, display, samsung, battery]"
7,0.083406,0.749806,0.083386,0.083402,1,tv cast,"[superb, buy, awesome, use]"
8,0.250000,0.250000,0.250000,0.250000,0,,"[super, wesome, price, camera]"
9,0.036963,0.327116,0.178526,0.457395,3,call samsung person buy online function,"[battery, backup, money, ok]"


#### again LDA -------------------------------------------------------

In [105]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['camera', 'display', 'person', 'm30s'],
 ['samsung', 'xcellent', 'buy', 'range'],
 ['quality', 'delivery', 'use', 'life'],
 ['battery', 'nice', 'superb', 'wid'],
 ['performance', 'network', 'super', 'handset']]

In [108]:
topics=print_top_words(lda,bow.get_feature_names(),4)
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=df["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]


(348, 1)

### BOW NMF  (results are better)

In [ ]:
bow=CountVectorizer(ngram_range=(1,3))
bow_review=bow.fit_transform(df["review"])
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
print_top_words(nmf,bow.get_feature_names(),8)

[['battery',
  'camera',
  'display',
  'camera battery',
  'battery life',
  'life',
  'mah',
  'display battery'],
 ['performance',
  'budget',
  'handset',
  'mode',
  'cost',
  'handset budget',
  'camera display',
  'processer'],
 ['display',
  'super',
  'xcellent',
  'ok',
  'camera',
  'smartphone',
  'camera display',
  'credit super'],
 ['camera',
  'quality',
  'wesome',
  'camera quality',
  'quality battery',
  'look',
  'osm',
  'love'],
 ['backup',
  'superb',
  'battery backup',
  'battery',
  'experience',
  'awesome',
  'picture quality',
  'camara'],
 ['samsung',
  'buy',
  'flipkart',
  'online',
  'm30s',
  'love',
  'buy online',
  'purchase'],
 ['money', 'value', 'value money', 'delivery', 'time', 'mark', 'hand', 'box'],
 ['use',
  'nice',
  'play',
  'bit',
  'ok',
  'battery pack',
  'month',
  'super month bit'],
 ['price',
  'range',
  'front',
  'price range',
  'performance',
  'battery',
  'camera',
  'excellent'],
 ['power',
  'team',
  'power button',
  

#### again NMF---------------------

In [ ]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['battery', 'backup', 'quality', 'battery backup'],
 ['camera', 'quality', 'm30s', 'samsung'],
 ['price', 'range', 'money', 'value'],
 ['ok', 'performance', 'button', 'power']]

### TFIDF LDA

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,3))
tfidf_words=tfidf.fit_transform(df["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,8)

[['love',
  'charge',
  'awesomespeechless',
  'cam',
  'voice',
  'battery good',
  'camera look',
  'good'],
 ['awesome',
  'range',
  'box',
  'nicc',
  'okay',
  'suprrrrrrrrr',
  'battery',
  'price range'],
 ['quality',
  'camera',
  'camera quality',
  'battery',
  'isplay',
  'thank',
  'smart',
  'purpose'],
 ['super', 'camera', 'samsung', 'nice', 'ok', 'date', 'function', 'worth'],
 ['xcellent',
  'smartphone',
  'perfomance',
  'excellent',
  'sm',
  'software',
  'anything',
  'waysome'],
 ['price',
  'battery',
  'mark',
  'wsm',
  'samsang',
  'till',
  'money good',
  'xcellent smartphone'],
 ['battery backup',
  'backup',
  'battery',
  'camera',
  'android',
  'perform',
  'worthless',
  'exellent'],
 ['money',
  'value',
  'value money',
  'time',
  'awsm',
  'flipkart',
  'light',
  'light camera'],
 ['samsung',
  'm30s',
  'sad',
  'mi',
  'samsung m30s',
  'samsung bro',
  'bro',
  'device'],
 ['wesome',
  'superb',
  'buy',
  'display',
  'buy online',
  'online',

#### Again LDF

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['android', 'display', 'm30s', 'awsm'],
 ['money', 'quality', 'backup', 'samsang'],
 ['perfomance', 'battery', 'love', 'samsang'],
 ['camera', 'time', 'charge', 'samsung']]

### TFIDF NMF

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,3))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [ ]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,6)

[['good', 'good samsung', 'good one', 'one', 'samsung', 'isplay good'],
 ['nice', 'phones', 'nice display', 'nice phones', 'nice awsm', 'osm nice'],
 ['wesome',
  'quality battery',
  'wesome camera',
  'battery',
  'wesome loving',
  'loving'],
 ['super', 'love', 'super nice', 'super camera', 'good love', 'super quality'],
 ['best', 'price', 'loved', 'good price', 'best price', 'samsung best'],
 ['superb',
  'great',
  'expected',
  'got',
  'got expected',
  'superb battery backup'],
 ['xcellent', 'money', 'flipkart', 'smartphone', 'thanks', 'excellent'],
 ['awesome', 'lovely', 'goods', 'thakarthu', 'exellent', 'worth money'],
 ['bad', 'ok', 'camera', 'camera quality', 'quality', 'worst'],
 ['battery', 'no', 'camera', 'battery backup', 'backup', 'display']]


#### again nmf -----------------

In [ ]:
arr=np.array(print_top_words(nmf,words,6)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'super', 'quality', 'wesome'],
 ['battery', 'backup', 'money', 'superb'],
 ['good', 'price', 'samsung', 'best'],
 ['nice', 'display', 'phones', 'got']]